In [ ]:
import torch
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Load the dataset
california = fetch_california_housing()
X, y = california.data, california.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create TensorDatasets
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

In [ ]:
import torch.nn as nn

class RNNRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(RNNRegressor, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x[:, -1, :])  # Using last sequence output
        return x

class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LSTMRegressor, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Using last sequence output
        return x

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for features, targets in train_loader:
            features = features.unsqueeze(1)  # Add sequence dimension
            optimizer.zero_grad()
            outputs = model(features).squeeze()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for features, targets in test_loader:
            features = features.unsqueeze(1)  # Add sequence dimension
            outputs = model(features).squeeze()
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(test_loader)

In [ ]:
input_dim = X_train.shape[1]
hidden_dim = 64

# Define the models, optimizers, and loss function
rnn_model = RNNRegressor(input_dim, hidden_dim)
rnn_optimizer = torch.optim.Adam(rnn_model.parameters())
rnn_criterion = nn.MSELoss()

lstm_model = LSTMRegressor(input_dim, hidden_dim)
lstm_optimizer = torch.optim.Adam(lstm_model.parameters())
lstm_criterion = nn.MSELoss()

# Train and evaluate RNN
train_model(rnn_model, train_loader, rnn_criterion, rnn_optimizer, num_epochs=50)
rnn_mse = evaluate_model(rnn_model, test_loader, rnn_criterion)  # Pass criterion here
print(f"RNN Test MSE: {rnn_mse}")

# Train and evaluate LSTM
train_model(lstm_model, train_loader, lstm_criterion, lstm_optimizer, num_epochs=50)
lstm_mse = evaluate_model(lstm_model, test_loader, lstm_criterion)  # Pass criterion here
print(f"LSTM Test MSE: {lstm_mse}")

Epoch 1, Loss: 0.7848940644153329
Epoch 2, Loss: 0.4881038557766944
Epoch 3, Loss: 0.47521566996682985
Epoch 4, Loss: 0.46073768360814615
Epoch 5, Loss: 0.440802519808906
Epoch 6, Loss: 0.42219798305873263
Epoch 7, Loss: 0.4081151943235088
Epoch 8, Loss: 0.3968458482237576
Epoch 9, Loss: 0.3880287832869289
Epoch 10, Loss: 0.3801048655503307
Epoch 11, Loss: 0.3732105886722489
Epoch 12, Loss: 0.367247852785585
Epoch 13, Loss: 0.3636733550802162
Epoch 14, Loss: 0.35805700771337334
Epoch 15, Loss: 0.3530645897258218
Epoch 16, Loss: 0.3492753896898366
Epoch 17, Loss: 0.34458413422324286
Epoch 18, Loss: 0.3408816324622739
Epoch 19, Loss: 0.33687162207973786
Epoch 20, Loss: 0.3361155196677807
Epoch 21, Loss: 0.3315263386570321
Epoch 22, Loss: 0.330221077492244
Epoch 23, Loss: 0.32795047430437085
Epoch 24, Loss: 0.32529158066857117
Epoch 25, Loss: 0.3251936472627486
Epoch 26, Loss: 0.32352246843321725
Epoch 27, Loss: 0.32232518376811414
Epoch 28, Loss: 0.32100682423356197
Epoch 29, Loss: 0.319